<a href="https://colab.research.google.com/github/ManuSinghYadav/Andrej_Karpathy_Zero_to_Hero/blob/main/Makemore_series_Lecture_4_(Backprop_Ninja).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## makemore: becoming a backprop ninja

In [ ]:
# there no change change in the first several cells from last lecture

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [ ]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2025-07-01 07:56:23--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.04s   

2025-07-01 07:56:23 (6.01 MB/s) - ‘names.txt’ saved [228145/228145]



In [ ]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [ ]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
# ok biolerplate done, now we get to the action:

In [ ]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [ ]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [ ]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [ ]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation (32, 30) @ (30, 64) --> (32, 64)
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3438, grad_fn=<NegBackward0>)

In [ ]:
logprobs[range(n), Yb]

tensor([-3.9640, -3.0388, -3.6712, -3.2018, -4.0355, -3.4311, -3.1464, -3.9840,
        -3.1977, -4.3773, -2.9947, -1.7152, -2.7312, -2.8632, -3.0243, -3.1010,
        -3.8401, -3.0077, -3.6139, -3.4498, -2.9115, -3.0200, -4.4247, -4.1560,
        -3.5304, -2.8540, -3.0613, -3.9470, -2.7314, -3.4725, -3.3274, -3.1777],
       grad_fn=<IndexBackward0>)

In [ ]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

dloss = 1

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = (-1.0/n) * dloss

dprobs = (1.0 / probs) * dlogprobs

# Now here we've to be very careful because here we have to see the shape of each
# as we're doing the element-wise multiplication and broadcasting might be
# happening.
counts.shape # (32, 27)
# And in element-wise ops where broadcasting is done, there are 2 ops are done,
# 1. Replication
# 2. Element-wise ops (+, -, *, /)
# So in backpropagation, whenever we're having a node is being used multiple
# times or replicating, we do sum of the gradients.

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim = True)
dcounts = counts_sum_inv * dprobs

dcounts_sum = -(1.0 / counts_sum**2) * dcounts_sum_inv

# counts_sum = counts.sum(1, keepdims=True)
# Here again check the shape, and gradients must be in shape of counts_sum
counts_sum.shape # (32, 1)
counts.shape # (32, 27)
# Something similar is happening here,
# a11 a12 a13 ---> b1 (= a11 + a12 + a13)
# a21 a22 a23 ---> b2 (= a21 + a22 + a23)
# So here we have to find the derivative of b's wrt to a's.
dcounts += torch.ones_like(counts) * dcounts_sum

dnorm_logits = counts * dcounts

# norm_logits = logits - logit_maxes
norm_logits.shape # (32, 27)
logits.shape # (32, 27)
logit_maxes.shape # (32, 1) # This is broadcasting

dlogits = 1.0 * dnorm_logits
dlogit_maxes = -(1.0 * dnorm_logits).sum(1, keepdim = True)

dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

# logits = h @ W2 + b2
# Here just check the sahpe of each with dlogits, as you have to multiply dlogits
# because of chain rule, so adjust them in such a way that they get multiplied.
# And the order comes in the same format
dlogits.shape # (32, 27)
h.shape # (32, 64)
W2.shape # (64, 27)
b2.shape # (27)
dh = dlogits @  W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

# h = torch.tanh(hpreact)
dhpreact = (1.0 - h**2) * dh

# hpreact = bngain * bnraw + bnbias
hpreact.shape # (32, 64)
bngain.shape # (1, 64) # Broadcasting is happening
bnraw.shape # (32, 64)
bnbias.shape # (1, 64)
dbngain = (bnraw * dhpreact).sum(0, keepdim = True)
dbnraw = bngain * dhpreact
dbnbias = (1.0 * dhpreact).sum(0, keepdim = True)

# bnraw = bndiff * bnvar_inv
bnraw.shape # (32, 64)
bndiff.shape # (32, 64)
bnvar_inv.shape # (1, 64) # Broadcasted
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim = True)

# bnvar_inv = (bnvar + 1e-5)**-0.5
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv

# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
# a11 a12
# a21 a22
# b1 = 1/n-1 * (a11 + a21)
# b2 = 1/n-1 * (a12 + a22)
bnvar.shape # (1, 64)
bndiff.shape # (32, 64)
dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# Here dbnvar will be broadcasted

# bndiff2 = bndiff**2
dbndiff += (2*bndiff) * dbndiff2

# bndiff = hprebn - bnmeani
bndiff.shape # (32, 64)
hprebn.shape # (32, 64)
bnmeani.shape # (1, 64) # Broadcasted
dhprebn = 1.0 * dbndiff
dbnmeani = (-1.0 * dbndiff).sum(0)

# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# hprebn = embcat @ W1 + b1
hprebn.shape # (32, 64)
embcat.shape # (32, 30)
W1.shape # (30, 64)
b1.shape # (64)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

# embcat = emb.view(emb.shape[0], -1)
embcat.shape # (32, 30)
emb.shape # (32, 3, 10)
demb = dembcat.view (emb.shape)

# emb = C[Xb]
emb.shape # (32, 3, 10)
C.shape # (27, 10)
Xb.shape # (32, 3)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
bngain          | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw   

In [ ]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 20000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
#with torch.no_grad():

# kick off optimization
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hprebn = embcat @ W1 + b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmean = hprebn.mean(0, keepdim=True)
  bnvar = hprebn.var(0, keepdim=True, unbiased=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = (hprebn - bnmean) * bnvar_inv
  hpreact = bngain * bnraw + bnbias
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  #loss.backward() # use this for correctness comparisons, delete it later!

  # manual backprop! #swole_doge_meme
  # -----------------
  dloss = 1
  # Cross entropy loss
  dlogits = F.softmax(logits, dim=1)
  dlogits[range(n), Yb] -= 1
  dlogits /= n

  # Linear layer 2
  dh = dlogits @ W2.T
  dW2 = h.T @ dlogits
  db2 = dlogits.sum(0)

  # Tanh
  dhpreact = (1.0 - h**2) * dh

  # BatchNorm layer
  # Backprop through hpreact = bngain * bnraw + bnbias
  dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
  dbnbias = dhpreact.sum(0, keepdim=True)
  dbnraw = bngain * dhpreact

  # Backprop through bnraw = (hprebn - bnmean) * bnvar_inv
  dbnvar_inv = ((hprebn - bnmean) * dbnraw).sum(0, keepdim=True) # Corrected
  dbndiff = bnvar_inv * dbnraw

  # Backprop through bnvar_inv = (bnvar + 1e-5)**-0.5
  dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv

  # Backprop through bnvar = hprebn.var(0, keepdim=True, unbiased=True)
  # Note: var(x) = mean((x - mean(x))**2), and unbiased=True means dividing by n-1
  bndiff = hprebn - bnmean # Recalculate bndiff for this batch
  dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff**2) * dbnvar # Corrected: use bndiff**2 for shape
  dbndiff += (2 * bndiff) * dbndiff2 # Add the gradient from variance calculation

  # Backprop through bnmean = hprebn.mean(0, keepdim=True)
  # The gradient of mean is 1/n * gradient(sum(x))
  # The gradient of bnmean also comes from the bndiff term in bnraw = (hprebn - bnmean) * bnvar_inv
  dbnmean = dbndiff.sum(0, keepdim=True) + (1.0/n) * dbndiff.sum(0, keepdim=True) # Corrected: Sum gradients from both paths

  # Backprop through bndiff = hprebn - bnmean
  dhprebn = dbndiff # Gradient from bndiff

  # Add the gradient from bnmean
  dhprebn += (1.0/n) * dbnmean # Corrected: Add the gradient of bnmean

  # Linear layer 1
  dembcat = dhprebn @ W1.T
  dW1 = embcat.T @ dhprebn
  db1 = dhprebn.sum(0)

  # Embeddings
  demb = dembcat.view (emb.shape)
  dC = torch.zeros_like(C)
  for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
      ix = Xb[k,j]
      dC[ix] += demb[k,j]

  # dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None # Delete this line
  grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
  # -----------------

  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p, grad in zip(parameters, grads):
    # p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward()) # Uncomment or remove as needed
    p.data += -lr * grad # new way of swole doge TODO: enable # Uncomment this line

  # track stats
  if i % 1000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  # if i >= 100: # TODO: delete early breaking when you're ready to train the full net
  #   break

12297
      0/  20000: 3.8078
   1000/  20000: 2.2315
   2000/  20000: 2.3222
   3000/  20000: 2.2265
   4000/  20000: 2.1772
   5000/  20000: 2.3468
   6000/  20000: 2.1933
   7000/  20000: 2.4198
   8000/  20000: 2.0917
   9000/  20000: 2.3729
  10000/  20000: 2.1800
  11000/  20000: 1.9365
  12000/  20000: 2.2651
  13000/  20000: 2.3382
  14000/  20000: 2.7276
  15000/  20000: 2.1705
  16000/  20000: 2.4674
  17000/  20000: 2.2992
  18000/  20000: 2.1224
  19000/  20000: 2.4413


In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [ ]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.277064800262451
val 2.284594774246216


In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

mria.
kayah.
keel.
ndhey.
alee.
mastendra.
gradeerdieli.
jeli.
jen.
edeison.
anar.
kayzioh.
karin.
sadbergihi.
jest.
jair.
jelin.
nopoorou.
zayven.
kyleni.
